# Map Dev for Reports

In [1]:
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *
from shared_utils import geography_utils

import altair as alt

import _data_cleaning
import _report_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [3]:
df = _report_utils.read_in_joined_data()

/home/jovyan/data-analyses/dla/atp/_report_utils.py:70: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [4]:
pd.set_option("display.max_columns",500)

In [5]:
mapsubset_cols= ['awarded','project_app_id', 'project_cycle', 'data_origin', 'geometry',
               'a1_imp_agcy_city','a1_imp_agcy_name','a1_proj_partner_agcy', 
               'assembly_district','congressional_district','senate_district',
              'a2_county', 'a2_ct_dist','a2_info_proj_name','a3_proj_type', 'total_atp_$', 'a2_proj_lat','a2_proj_long']

In [6]:
df_map = _report_utils.fix_geom_issues(df, mapsubset_cols)

In [7]:
#df_map.explore("data_origin", cmap="tab20b")

In [8]:
## from amanda's notebook for Project prioritization: https://github.com/cal-itp/data-analyses/blob/main/project_prioritization/_utils.py
shapes = "https://gis.data.ca.gov/datasets/0144574f750f4ccc88749004aca6eb0c_0.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
district_shapes = gpd.read_file(f"{shapes}").to_crs(epsg=4326)   

In [9]:
#district_shapes.boundary.explore()

In [10]:
## county geography

shapes2 = "https://opendata.arcgis.com/datasets/8713ced9b78a4abb97dc130a691a8695_0.geojson"
county_shapes = gpd.read_file(f"{shapes2}").to_crs(epsg=4326)

In [11]:
#county_shapes.boundary.explore()

In [12]:
df_map = df_map.drop(columns='index_right')

## Joining District and Points

In [56]:
def nunique_by_geography(df,
                         geodf,
                         geog_col,
                         agg_col,
                         geodf_mergeon_col):
    
    sum_df = (df>>group_by(_[geog_col])
              >>summarize(n_unique = _[agg_col].nunique()))
    
    joined_df = geodf.merge(sum_df, 
                            how='inner',
                            left_on=[geodf_mergeon_col],
                            right_on=[geog_col])
    
    return joined_df
              

In [58]:
by_dist = nunique_by_geography(df_map,
                         district_shapes,
                         'a2_ct_dist',
                         'project_app_id',
                         'DISTRICT')

In [59]:
#by_dist.explore('n_applications', cmap='Oranges', highlight=True, legend=True)

## Layering

In [17]:
#base = district_shapes.plot(color='white', edgecolor='black')

#df_map.plot(ax=base, marker='o', color='blue', markersize=5)

In [21]:
# base = district_shapes.plot(figsize=(8,8), color='white', edgecolor='black')
# layered = df_map.plot(ax=base, marker='o', legend='true', column='data_origin', markersize=10)
# layered.set_title('ATP Applications',fontsize=18)

# layered

In [19]:
import folium

In [46]:
## following map works, commenting out to save

In [47]:
# m = by_dist.explore(
#      column="n_applications", 
#      legend=True,
#     color = 'oranges',
#      legend_kwds=dict(colorbar=True),
#      tooltip=["District", "Number of Applications"], 
#      name="Number of Applications by County")

# df_map.explore(
#      m=m, 
#      color="Data Origin", 
#      marker_kwds=dict(radius=3, fill=True),
#      tooltip=["Data Origin", "Project Cycle", "Implemeting Agency", "Project Name"], 
#      tooltip_kwds=dict(labels=True),
#      name="Project Locations"
# )

# folium.TileLayer('Stamen Toner', control=True).add_to(m) 
# folium.LayerControl().add_to(m)

# m

In [51]:
# points = df_map.explore(
#      column="data_origin", 
#      legend=True,
#      legend_kwds=dict(colorbar=False),
#      name="Application Result",
#      tooltip=["data_origin", "project_cycle", "a1_imp_agcy_city", "a2_info_proj_name"], 
#      tooltip_kwds=dict(labels=False), 
# )

In [52]:
# points

In [62]:
## By Congressional Dist

In [61]:
df_map.sample()

,project_app_id,project_cycle,awarded,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_ct_dist,a2_info_proj_name,a2_proj_lat,a2_proj_long,a3_proj_type,total_atp_$,State,point_check
227,"10-Lathrop,City of-1",5,N,Application,POINT (-121.29787 37.80354),Lathrop,"Lathrop,City of",NaN,12,9,5,San Joaquin,10.00,Class II Bikeway to ACE Station,37.80,-121.30,Infrastructure - Small,NaN,CA,Point In State


In [ ]:
nunique_by_geography(df,
                         geodf,
                         geog_col,
                         agg_col,
                         geodf_mergeon_col)

In [63]:
## By Senate Dist

In [64]:
## By Assembly Dist